In [18]:
import sqlalchemy
import pandas as pd
from sentence_transformers import SentenceTransformer, util
import pathlib2 as pathlib
import numpy as np
import json
from sqlalchemy import create_engine

In [19]:
cwd_path=pathlib.Path().cwd()
data_path=cwd_path.joinpath('wirtschaftsinformatik-main')
db_file=cwd.joinpath('cards.db')

In [3]:
# transformer=SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')
# model = SentenceTransformer('paraphrase-MiniLM-L12-v2')
def get_tensor(sentence):
    transformer=SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')
    return transformer.encode(sentence,convert_to_tensor=True)
# def encode_questions(df):
#     df['q_tensor']=df.q.transform(get_tensor)
#     return df
# def encode_answers(df):
#     df['a_tensor']=df.a.transform(get_tensor)
#     return df
def read_json_files(deck_files):
    frames=[]
    for file in deck_files:
    #     print(file)
        topic=file.stem
        with open(file,encoding='utf-8') as f:
            data = json.load(f)
        decks=data['decks']
        for deck in decks:
    #         print(deck)
            cards=decks[deck]['cards']
            df=pd.DataFrame(cards)
            df=df.transpose()
            df=df.assign(topic=[topic for x in range(len(df.index))],deck=[deck for x in range(len(df.index))])
    #         df['topic']=pd.Series([topic for x in range(len(df.index))], index=df.index)
    #         df['deck']=pd.Series([deck for x in range(len(df.index))], index=df.index)
            df['q_tensor']=df.q.transform(get_tensor)
            df['a_tensor']=df.a.transform(get_tensor)
            frames.append(df)
            print('deck finished:',deck)
        print('topic finished:',topic)
    dataframe=pd.concat(frames,axis=0)
    return dataframe

In [12]:
deck_files=list(data_path.glob(r'*.json'))

In [20]:
data_path

WindowsPath('C:/Users/lekle/Documents/Projects/Virtual_Camera_Project/wirtschaftsinformatik-main')

In [4]:
deck_files=list(data_path.glob(r'*.json'))
dataframe=read_json_files(deck_files)

ValueError: No objects to concatenate

In [ ]:
engine = create_engine('sqlite:///'+db_file.as_posix(), echo=False)

dataframe['q_tensor']=dataframe.q_tensor.transform(lambda x: x.tolist())
dataframe['a_tensor']=dataframe.q_tensor.transform(lambda x: x.tolist())

dataframe = dataframe.applymap(str)

dataframe.to_sql('cards', con=engine,if_exists='replace')
dataframe.to_csv('data.csv',sep=';',index=False)